# TP2 MLNLP

In [1]:
import numpy as np

In [ ]:
#python -m spacy download fr_core_news_sm
import spacy
import re

nlp = spacy.load("fr_core_news_sm")

In [ ]:
with open("Tao_Te_King.txt") as f:
    txt_raw = f.read()
txt = re.sub("\n{2,}","\n",txt_raw)

phrases = txt.split("\n")
sphrases = [nlp(p) for p in phrases]
type(sphrases[0])

# Enlever la fin des phrases

In [ ]:
print(sphrases[1])

In [ ]:
len(sphrases[1])

In [ ]:
print(sphrases[11])

In [8]:
sphrase = sphrases[11]
print(sphrase, len(sphrase))
i = 1
for token in sphrase:
    print(i, token.text)
    i+=1

Dans le monde, lorsque tous les hommes ont su apprÃ©cier la beautÃ© (morale), alors la laideur (du vice) a paru. 30
1 Dans
2 le
3 monde
4 ,
5 lorsque
6 tous
7 les
8 hommes
9 ont
10 su
11 apprÃ
12 ©
13 cier
14 la
15 beautÃ
16 ©
17 (
18 morale
19 )
20 ,
21 alors
22 la
23 laideur
24 (
25 du
26 vice
27 )
28 a
29 paru
30 .


# Questions

1. Quel est le vocabulaire de ce corpus ? Taille ? Combien de mots de plus de 5 occurrences ? Quels sont les mots les plus fréquents ? Les moins fréquents ?
2. Extrayez le dictionnaire de bigrammes (Pensez à ajouter des tokens `<s>` et `</s>` en début et en fin de phrase/séquence). Combien de bigrammes ? Les + fréquents ? Les - fréquents ?
3. Découper le corpus en train/test (90%/10%). Estimer les proba bi-gram sur train.
Calculer les log-proba des phrases de test. Calculer la perplexité. Comparer à la perplexité du modèle unigramme.
4. Générer en utilisant les modèles de langages (une ? plusieurs ? phrases)

In [9]:
sphrase = sphrases[11]
for token in sphrase:
    print(token.text, token.pos_)

Dans ADP
le DET
monde NOUN
, PUNCT
lorsque SCONJ
tous ADJ
les DET
hommes NOUN
ont AUX
su VERB
apprÃ PROPN
© PROPN
cier VERB
la DET
beautÃ NOUN
© PROPN
( PUNCT
morale NOUN
) PUNCT
, PUNCT
alors ADV
la DET
laideur NOUN
( PUNCT
du ADP
vice NOUN
) PUNCT
a AUX
paru VERB
. PUNCT


### Question 1
1. Quel est le vocabulaire de ce corpus ? Taille ? Combien de mots de plus de 5 occurrences ? Quels sont les mots les plus fréquents ? Les moins fréquents ?

In [10]:
import collections

In [11]:
for p in sphrases[1:-1] :
    c = collections.Counter(p)

In [12]:
len(nlp.vocab)

2303

In [13]:
txt_spaces = re.sub('\n', ' ', txt)

In [14]:
smots = nlp(txt_spaces)

Vocabulaire du corpus

In [15]:
vocabulaire = dict()
for phrase in sphrases:
    for token in phrase:
        t = token.text
        if t in vocabulaire:
            vocabulaire[t] += 1
        else:
            vocabulaire[t] = 1
vocabulaire

{'LAO': 1,
 'TSEU': 1,
 'TAO': 1,
 'TE': 1,
 'KING': 1,
 'LE': 1,
 'LIVRE': 3,
 'DE': 2,
 'LA': 2,
 'VOIE': 1,
 'ET': 1,
 'VERTU': 1,
 'I.': 2,
 'CHAPITRE': 81,
 'La': 12,
 'voie': 6,
 'qui': 177,
 'peut': 33,
 'Ãªtre': 62,
 'exprimÃ': 1,
 '©': 626,
 'e': 25,
 'par': 27,
 'la': 224,
 'parole': 3,
 'nâ€': 96,
 '™': 794,
 'est': 325,
 'pas': 130,
 'Voie': 5,
 'Ã': 287,
 'ternelle': 5,
 ';': 133,
 'le': 307,
 'nom': 12,
 'nommÃ': 2,
 'Nom': 1,
 'ternel': 4,
 '.': 708,
 '(': 110,
 'Lâ€': 22,
 ')': 110,
 'sans': 19,
 'lâ€': 228,
 'origine': 12,
 'du': 86,
 'ciel': 31,
 'et': 242,
 'de': 309,
 'terre': 16,
 'avec': 18,
 'un': 88,
 ',': 411,
 'il': 226,
 'mÃ¨re': 8,
 'toutes': 8,
 'choses': 30,
 'Câ€': 68,
 'pourquoi': 68,
 'lorsquâ€': 10,
 'on': 108,
 'constamment': 12,
 'exempt': 4,
 'passions': 5,
 'voit': 6,
 'son': 34,
 'essence': 3,
 'spirituelle': 5,
 'a': 71,
 'des': 68,
 'sous': 1,
 'une': 83,
 'forme': 6,
 'bornÃ': 4,
 'e.': 4,
 'Ces': 2,
 'deux': 10,
 'ont': 28,
 'mÃªme': 18,
 'reÃ

In [16]:
print("Taille du vocabulaire :", len(vocabulaire))

Taille du vocabulaire : 1898


In [17]:
voc_frequent = {k:v for (k,v) in vocabulaire.items() if v > 5}

In [18]:
print("Nombre des mots ayant des occurrences > 5 :", len(voc_frequent))

Nombre des mots ayant des occurrences > 5 : 269


Mots les plus fréquents :

In [19]:
sorted(vocabulaire.items(), key=lambda item: item[1])[-10:]

[('les', 233),
 ('et', 242),
 ('Ã', 287),
 ('le', 307),
 ('de', 309),
 ('est', 325),
 (',', 411),
 ('©', 626),
 ('.', 708),
 ('™', 794)]

Mots les moins fréquents: 

In [20]:
voc_moins_freq = {k:v for (k,v) in vocabulaire.items() if v == 1}

In [21]:
print("Nombre de mots ayant 1 occurrence :", len(voc_moins_freq))

Nombre de mots ayant 1 occurrence : 1024


In [22]:
sorted(vocabulaire.items(), key=lambda item: item[1])[:10]

[('LAO', 1),
 ('TSEU', 1),
 ('TAO', 1),
 ('TE', 1),
 ('KING', 1),
 ('LE', 1),
 ('VOIE', 1),
 ('ET', 1),
 ('VERTU', 1),
 ('exprimÃ', 1)]

### Question 2

2. Extrayez le dictionnaire de bigrammes (Pensez à ajouter des tokens `<s>` et `</s>` en début et en fin de phrase/séquence). Combien de bigrammes ? Les + fréquents ? Les - fréquents ?

In [23]:
sphrases_list = []
for phrase in sphrases:
    phrase_list = ['<s>']
    for token in phrase:
        phrase_list.append(token.text)
    phrase_list.append("</s>")
    sphrases_list.append(phrase_list)

In [24]:
unigramme_dict = dict()
for phrase in sphrases_list : 
    for token_i in range(len(phrase)-1) :
        elem  = phrase[token_i]
        if elem in unigramme_dict:
            unigramme_dict[elem] += 1
        else:
            unigramme_dict[elem] = 1

In [25]:
unigramme_dict["Pourquoi"]

2

In [26]:
bigramme_dict = dict()
for phrase in sphrases_list : 
    for token_i in range(len(phrase)-1) :
        elem  = phrase[token_i]+" "+phrase[token_i+1]
        if elem in bigramme_dict:
            bigramme_dict[elem] += 1
        else:
            bigramme_dict[elem] = 1

In [27]:
print("Nombre de bigrammes :", len(bigramme_dict))

Nombre de bigrammes : 6316


Bigrammes les + fréquents :

In [28]:
sorted(bigramme_dict.items(), key=lambda item: item[1])[-1:-10:-1]

[('. </s>', 676),
 ('lâ€ ™', 228),
 ('Ã \xa0 ', 153),
 ('™ est', 140),
 ('Ã ©', 126),
 ('sâ€ ™', 105),
 ('nâ€ ™', 96),
 ('dÃ ©', 85),
 ('dâ€ ™', 83)]

Bigrammes les - fréquents :

In [29]:
sorted(bigramme_dict.items(), key=lambda item: item[1])[:10]

[('<s> LAO', 1),
 ('LAO TSEU', 1),
 ('TSEU TAO', 1),
 ('TAO TE', 1),
 ('TE KING', 1),
 ('KING </s>', 1),
 ('<s> LE', 1),
 ('LE LIVRE', 1),
 ('LIVRE </s>', 1),
 ('<s> DE', 1)]

In [30]:
bigramme_moins_freq = {k:v for (k,v) in bigramme_dict.items() if v == 1}

In [31]:
print("Nombre de mots ayant 1 occurrence :", len(bigramme_moins_freq))

Nombre de mots ayant 1 occurrence : 4539


### Question 3
3. Découper le corpus en train/test (90%/10%). Estimer les proba bi-gram sur train.
Calculer les log-proba des phrases de test. Calculer la perplexité. Comparer à la perplexité du modèle unigramme.

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
train_phrases, test_phrases = train_test_split(sphrases_list, test_size=0.1, random_state=42)

In [34]:
train_phrases

[['<s>', 'CHAPITRE', 'XLIX', '.', '</s>'],
 ['<s>',
  'Les',
  'choses',
  'grandes',
  'ou',
  'petites',
  ',',
  'nombreuses',
  'ou',
  'rares',
  ',',
  '(',
  'sont',
  'Ã',
  '©',
  'gales',
  'Ã',
  '\xa0 ',
  'ses',
  'yeux',
  ')',
  '.',
  '</s>'],
 ['<s>',
  'Il',
  'se',
  'plaÃ',
  '®',
  't',
  'dans',
  'la',
  'situation',
  'la',
  'plus',
  'humble',
  '.',
  '</s>'],
 ['<s>',
  'Celui',
  'qui',
  'excelle',
  'Ã',
  '\xa0 ',
  'commander',
  'une',
  'armÃ',
  '©',
  'e',
  ',',
  'nâ€',
  '™',
  'a',
  'pas',
  'une',
  'ardeur',
  'belliqueuse',
  '.',
  '</s>'],
 ['<s>',
  'Tous',
  'les',
  'Ãªtres',
  'fuient',
  'le',
  'calme',
  'et',
  'cherchent',
  'le',
  'mouvement',
  '.',
  '</s>'],
 ['<s>',
  'Il',
  'est',
  'Ã',
  '©',
  'ternel',
  'et',
  'semble',
  'exister',
  '(',
  'matÃ',
  '©',
  'riellement',
  ')',
  '.',
  '</s>'],
 ['<s>',
  'Câ€',
  '™',
  'est',
  'pourquoi',
  'celui',
  'qui',
  'a',
  'de',
  'grandes',
  'passions',
  'est',
  '

Unigrammes

In [35]:
unigramme_dict_train = dict()
for phrase in train_phrases : 
    for token_i in range(len(phrase)-1) :
        elem  = phrase[token_i]
        if elem in unigramme_dict_train:
            unigramme_dict_train[elem] += 1
        else:
            unigramme_dict_train[elem] = 1

Bigrammes

Probas

In [36]:
bigramme_dict_train = dict()
for phrase in train_phrases : 
    for token_i in range(len(phrase)-1) :
        elem  = phrase[token_i]+" "+phrase[token_i+1]
        if elem in bigramme_dict_train:
            bigramme_dict_train[elem] += 1
        else:
            bigramme_dict_train[elem] = 1

In [37]:
sum(bigramme_dict_train.values())

13368

Calcul de probabilités : 
$$
P(W_k|W_{k-1}) = \frac{|W_{k-1}W_k|}{|W_{k-1}|}
$$

In [38]:
mot = "<s> je"
mot.split()


['<s>', 'je']

In [39]:
bigram_train_probas = dict()
for bigram in bigramme_dict_train.keys():
    premier_mot = bigram.split()[0]
    bigram_train_probas[bigram] = bigramme_dict_train[bigram]/unigramme_dict_train[premier_mot]

Log-probas de l'ensemble de test : le logarithme des probas.

Méthode (je pense qu'elle est mauvaise) : J'ai calculé les occurrences dans l'ensemble de test, mais je pense qu'il faut prendre les occurrences précédentes ?

Unigrammes de l'ensemble test : 

In [40]:
unigramme_dict_test = dict()
for phrase in test_phrases : 
    for token_i in range(len(phrase)-1) :
        elem  = phrase[token_i]
        if elem in unigramme_dict_test:
            unigramme_dict_test[elem] += 1
        else:
            unigramme_dict_test[elem] = 1

Bigrammes

In [41]:
bigramme_dict_test = dict()
for phrase in test_phrases : 
    for token_i in range(len(phrase)-1) :
        elem  = phrase[token_i]+" "+phrase[token_i+1]
        if elem in bigramme_dict_test:
            bigramme_dict_test[elem] += 1
        else:
            bigramme_dict_test[elem] = 1

In [42]:
np.log(12)

2.4849066497880004

In [46]:
len(list(bigramme_dict_test.keys())) == len(set(bigramme_dict_test.keys()))

True

In [43]:
bigram_test_log_probas = []
for bigram in bigramme_dict_test.keys():
    premier_mot = bigram.split()[0]
    if bigram not in bigramme_dict_train:
        bigramme_dict_train[bigram] = 0
    if not 
    bigram_test_log_probas.append(np.log(bigramme_dict_train[bigram]+1/(unigramme_dict_train[premier_mot]+len()))

C:\Users\x-app\AppData\Local\Temp\ipykernel_11556\2269090734.py:6: RuntimeWarning: divide by zero encountered in log
  bigram_test_log_probas.append(np.log(bigramme_dict_train[bigram]/unigramme_dict_train[premier_mot]))


KeyError: 'droits'

Pour la perplexité : 
$ \sqrt[n]{\frac{1}{p}} = p^{-\frac{1}{n}}. exp(-\frac{1}{n}logp) $

Calcul de perplexité : 

Pareil, je pense ne pas faire le bon calcul. Par exemple, le n peut ne être pas le bon, les probas : il y a plusieurs fois le même bigramme, qui ici ne sera compté qu'une fois ? Par contre moodle est down, je ne pourrais pas aller voir le pdf avant demain.

In [ ]:
np.exp((-1/(np.sum(list(unigramme_dict_test.values()))))*(np.sum(bigram_test_log_probas)))

Le résultat n'est pas bon. Il faut revoir tous les problèmes mentionnés ci-dessus. Un 3e problème : je fais la différence pour la casse. "Pourquoi" et "pourquoi" sont comptés comme étant différents, par exemple. Je vais faire chaque problème, en commençant par les probas individuelles. 

* Avec probas individuelles : Ne fonctionne pas. Renvoie 2.68.

* En prenant les probas de train : il faut faire le smoothing, sinon on a des probas à 0. Je vais réfléchir à cela.

* On prend uniquement les probas, et pas les occurrences qui vont avec. Ce n'est pas bon. Il faut faire occ*logproba.